In [13]:
import os, sys, re
import pandas as pd
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
from datetime import datetime, timedelta

current_directory = os.getcwd()
root_directory = os.path.abspath(os.path.join(current_directory, os.pardir))
sys.path.append(root_directory)

* **Concate all data**

In [14]:
import os
import pandas as pd

# Directory containing the CSV files
#directory = r'c:\Users\mohammed\OneDrive\Documents\QFM -S2\BankReviewIntelligence\ScrapperService\production_standalone\row_data\parcket\2024-06-08'
directory = "../../ScrapperService/production_standalone/row_data/parcket/2024-06-08"
# List to hold the data from each CSV file
data_frames = []

# Iterate over all files in the directory
for country in os.listdir(directory):
    if country != ".DS_Store":
        for city in os.listdir(directory+"/"+country):
            if city.endswith(".parquet"):
                file_path = os.path.join(directory+"/"+country, city)
                df = pd.read_parquet(file_path)
                data_frames.append(df)

# Concatenate all DataFrames
all_data = pd.concat(data_frames, ignore_index=True)
all_data['Reviewer_Publish_Date'] = all_data['Reviewer_Publish_Date'].str.replace('\xa0', ' ')

# Save the concatenated DataFrame to a new CSV file
all_data.to_csv('concatenated_data.csv', index=False)
print(all_data.shape)
#all_data.head()

(1710, 14)


* **Preprocessing**

In [4]:
from parquet_preprocessing import preprocess_dataframe
all_data = pd.read_parquet('concatenated_data.parquet')

In [6]:
df = preprocess_dataframe(all_data)

In [10]:
df

,Country,Town,Bank_Name,Bank_Phone_number,Bank_Address,Bank_Website,Reviewer_Nane,Reviewer_Sart,Reviewer_Text,Reviewer_Publish_Date,Reviewer_Like_Reaction,Reviewer_Profil_Link,Reviewer_Owner_Reply,Reviewer_Owner_Reply_Date,Topic,Sentiment,Sub_Topic
0,Kenya,Nyeri,Standard Chartered Bank Nyeri Branch,+254 20 3293900,"Kenyatta Rd, Nyeri, Kenya",standardchartered.com,V M,5,NAN,2018-06-29,0,https://www.google.com/maps/contrib/1169461831...,None,0000-00-00,NaN,NaN,NaN
1,Kenya,Kitui,Co-Op Bank Atm - Kitui Branch,+254 44 4422370,"Emco House, Kilungya Makuti Street, Kitui, Kenya",co-opbank.co.ke,Emman Emmah,5,NAN,2023-10-31,0,https://www.google.com/maps/contrib/1125754277...,None,0000-00-00,NaN,NaN,NaN
2,Kenya,Voi,Kcb Voi,Not available,"Voi, Kenya",not available,Isaac Akuru Miruka,5,NAN,2021-06-28,0,https://www.google.com/maps/contrib/1090182152...,None,0000-00-00,NaN,NaN,NaN
3,Kenya,Kitale,Standard Chartered Bank - Kitale Branch,+254 20 3293900,"Ground Floor- Teryet Business Centre, Kenyatta...",standardchartered.com,Tony Tonny,1,NAN,2020-06-28,0,https://www.google.com/maps/contrib/1018542844...,None,0000-00-00,NaN,NaN,NaN
4,Kenya,Homa Bay,"Barclays Bank, Homa Bay",Not available,"Fff4+7W4, Homa Bay, Kenya",not available,Declan Ottaro,3,Salle bancaire,2018-06-29,0,https://www.google.com/maps/contrib/1023346861...,None,0000-00-00,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1705,Côte D'Ivoire,Divo,Sgci,+225 51225122,"Rjrr+4W8, Divo, Côte D’Ivoire",not available,Marie Germaine,5,NAN,2019-06-29,0,https://www.google.com/maps/contrib/1054373247...,None,0000-00-00,NaN,NaN,NaN
1706,Côte D'Ivoire,Divo,Coopec Divo,Not available,"Rjmm+P5W, Divo, Côte D’Ivoire",not available,Comoe Djibril,2,NAN,2019-06-29,0,https://www.google.com/maps/contrib/1132578295...,None,0000-00-00,NaN,NaN,NaN
1707,Côte D'Ivoire,Divo,Coopec Divo,Not available,"Rjmm+P5W, Divo, Côte D’Ivoire",not available,Comoe Djibril,2,NAN,2019-06-29,0,https://www.google.com/maps/contrib/1132578295...,None,0000-00-00,NaN,NaN,NaN
1708,Côte D'Ivoire,Divo,Hamed & El Djo Transfert Divo,+225 05 56 33 4482,"Bp 385, Divo, Côte D’Ivoire",not available,Moctar Kone (Kmoctar),3,NAN,2023-06-28,0,https://www.google.com/maps/contrib/1036178797...,None,0000-00-00,NaN,NaN,NaN


* **Topic generation**